In [1]:
def decorated_by(func):
    func.__doc__ += '\nDecorated by decorated_by.'
    return func

In [2]:
def add(x, y):
    """Return the sum of x and y."""
    return x + y

In [3]:
add = decorated_by(add)

In [6]:
help(add)

Help on function add in module __main__:

add(x, y)
    Return the sum of x and y.
    Decorated by decorated_by.



In [5]:
@decorated_by
def add(x, y):
    """Return the sum of x and y."""
    return x + y

In [7]:
@also_decorated_by
@decorated_by
def add(x, y):
    """Return the sum of x and y."""
    return x + y


NameError: name 'also_decorated_by' is not defined

In [8]:
registry = []
def register(decorated):
    registry.append(decorated)
    return decorated

In [9]:
@register
def foo():
    return 3

In [10]:
@register
def bar():
    return 5

In [11]:
bar()

5

In [12]:
answers = []
for func in registry:
    answers.append(func())

In [13]:
answers

[3, 5]

In [14]:
class Registry(object):
    def __init__(self):
        self.__functions = []
    def register(self, decorated):
        self.__functions.append(decorated)
        return decorated

    def run_all(self, *args, **kwargs):
        return_values = []
        for func in self.__functions:
            return_values.append(func(*args, **kwargs))
        return return_values

In [15]:
a = Registry()
b = Registry()

In [18]:
@a.register
def foo(x=3):
    return x

In [24]:
@b.register
def bar(x=5):
    return x

In [21]:
@a.register
@b.register
def baz(x=7):
    return x

In [22]:
a.run_all()

[3, 7]

In [25]:
b.run_all()

[7, 5]

In [27]:
a.run_all(4)

[4, 4]

In [28]:
def requires_ints(decorated):
    def inner(*args, **kwargs):
        # Get any values that may have been sent as keyword arguments.
        kwarg_values = [i for i in kwarg_values()]

        # Iterate over every value sent to the decorated method,
        # and ensure that each one is an integer; raise TypeError if not
        for arg in list(args) + kwarg_values:
            if not isinstance(arg, int):
                raise TypeError('%s only accepts integers as arguments.' % decorated.__name)
        # Run the decorated method, and return the result
        return decorated(*args, **kwargs)
    return inner

In [29]:
@requires_ints
def foo(x, y):
    """Return the sum of x and y."""
    return x + y


In [30]:
help(foo)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [31]:
import functools

def requires_ints(decorated):
    @functools.wraps(decorated)
    def inner(*args, **kwargs):
        # Get any values that may have been sent as keyword args.
        kwarg_values = [i for i in kwarg_values()]

        # Iterate over every value sent to the decorated method, and
        # ensure that each one is an integer; raise TypeError if not
        for arg in args + kwarg_values:
            if not isinstance(i, int):
                raise TypeError('%s only accepts integers as arguments.' % decorated.__name__)

        # Run the decorated method, and return the result. 
        return decorated(*args, **kwargs)
    return inner

In [32]:
@requires_ints
def foo(x, y):
    """Return the sum of x and y."""
    return x + y

In [33]:
help(foo)

Help on function foo in module __main__:

foo(x, y)
    Return the sum of x and y.



In [ ]:
class User(object):
    """A representation of a user in our application."""

    def __init__(self, username, email):
        self.username = username
        self.email = email

class AnonymousUser(User):
    """An anonymous user; a stand-in for an actual user that none
    theless is not an actual user."""
    def __init__(self):
        self.username = None
        self.email = None
    
    def __nonzero__(self):
        return False

In [34]:
import functools

def requires_user(func):
    @functools.wraps(func)
    def inner(user, *args, **kwargs):
        """Verify that the user is truthy;
        if so, run the decorated method,
        and if not, raise ValueError"""
        # Ensure that user is truthy, and of correct type
        # The 'truthy' check will fail on annonymous users, since
        # the AnonymousUser subclass has a '__nonzero__' method
        # that returns False.
        if user and isinstance(user, User):
            return func(user, *args, **kwargs)
        else:
            raise ValueError('A Valid user is required to run this.')
    return inner

In [39]:
import functools
import json

def json_output(decorated):
    """Run the decorated function, serialize the result of that function
    to JSON, and return the JSON string."""
    @functools.wraps(decorated)
    def inner(*args, **kwargs):
        result = decorated(*args, **kwargs)
        return json.dumps(result)
    return inner


In [40]:
@json_output
def do_nothing():
    return {'status': 'done'}

In [41]:
do_nothing()


'{"status": "done"}'

In [42]:
emy = do_nothing()
type(emy)

str

In [44]:
import functools
import json

class JSONOutputError(Exception):
    def __init__(self, message):
        self._message = message

    def __str__(self):
        return self._message

def json_output(decorated):
    """Run the decorated function, serialise the result of that
    function to JSON and return the JSON string."""
    @functools.wraps(decorated)
    def inner(*args, **kwargs):
        try:
            result = decorated(*args, **kwargs)
        except JSONOutputError as ex:
            result = {
                'status': 'error',
                'message': str(ex),
            }
        return json.dumps(result)
    return inner
            

In [45]:
@json_output
def error():
    raise JSONOutputError('This function is erratic.')

In [46]:
error()

'{"status": "error", "message": "This function is erratic."}'

In [47]:
@json_output
def other_error():
    raise ValueError('The grass is always greener...')

In [48]:
other_error()

ValueError: The grass is always greener...

In [54]:
import functools
import logging
import time

def logged(method):
    """Cause the decorated method to be run and its results logged,
    along with some other diagnostic information."""
    @functools.wraps(method)
    def inner(*args, **kwargs):
        # Record our start time
        start = time.time()

        # Run the decorated method
        return_value = method(*args, **kwargs)

        # Record our completion time, and calculate the delta
        end = time.time()
        delta = end - start

        # Log the method call and the result
        logger = logging.getLogger('decorator.logged')
        logger.warn('Called method %s at %.02f; execution time %.02f ' 
        'seconds; result %r.' %
        (method.__name__, start, delta, return_value))

        # Return the method's original return value.
        return return_value
    return inner 

In [55]:
import time
@logged
def sleep_and_return(return_value):
    time.sleep(2)
    return return_value


In [56]:
sleep_and_return(42)

Called method sleep_and_return at 1597771537.33; execution time 2.00 seconds; result 42.


42

In [58]:
import functools
import json

class JSONOutputError(Exception):
    def __init__(self, message):
        self.message = message

    def __str__(self):
        return self.message

def json_output(indent=None, sort_keys=False):
    """Run the decorated function, serialize the result of that function
    to JSON, and return the JSON string.
    """
    def actual_decorator(decorated):
        @functools.wraps(decorated)
        def inner(*args, **kwargs):
            try:
                result = decorated(*args, **kwargs)
            except JSONOutputError as ex:
                result = {
                    'status': 'error',
                    'message': str(ex),
                 }
            return json.dumps(result, indent=indent, sort_keys=sort_keys)
        return inner
    return actual_decorator


In [60]:
@json_output(indent=4)
def do_nothing():
    return {'status':'done'}

In [61]:
do_nothing()

'{\n    "status": "done"\n}'

In [62]:
import functools
import json

class JSONOutputError(Exception):
    def __init__(self, message):
        self._message = message

    def __str__(self):
        return self._message

def json_output(decorated_=None, indent=None, sort_keys=False):
    """Run the decorated function, serialize the result of that function
    to JSON, and return the JSON string."""
    # Did we get both a decorated method and keyword args?
    # That should not happen.
    if decorated_ and (indent or sort_keys):
        raise RuntimeError('Unexpected arguments.')

    # Define the actual decorator function.
    def actual_decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            try:
                result = func(*args, **kwargs)
            except JSONOutputError as ex:
                result = {
                    'status': 'error',
                    'message': str(ex),
                }
            return json.dumps(result, indent=indent, sort_keys=sort_keys)
        return inner
    # Return either the actual decorator, or the result of applying
    # the actual decorator, depending on what arguments we got.
    if decorated_:
        return actual_decorator(decorated_)
    else:
        return actual_decorator


In [63]:
import functools
import time

def sortable_by_creation_time(cls):
    """Given a class, augment the class to have its instances be 
    sortable by the timestamp at which they were instantiated."""
    # Augment the class 'original '__init__' method to also store a
    # '_created_' attribute on the instance, which corresponds to 
    # when it was instantiated
    original_init = cls.__init__

    @functools.wraps(original_init)
    def new_init(self, *args, **kwargs):
        original_init(self, *args, **kwargs)
        self._created = time.time()
    cls.__init__ = new_init

    # Add '__lt__' and '__gt__' methods that return True or False
    # based on the created values in question
    cls.__lt__ = lambda self, other: self._created < other._created
    cls.__gt__ = lambda self, other: self._created > other.created

    # Done; return the class object
    return cls


In [64]:
@sortable_by_creation_time
class Sortable(object):
    def __init__(self, identifier):
        self.identifier = identifier

    def __repr__(self):
        return self.identifier

In [65]:
first = Sortable('first')

In [66]:
second = Sortable('second')

In [67]:
third = Sortable('third')

In [68]:
sortables = [second, first, third]

In [69]:
sorted(sortables)

[first, second, third]

In [70]:
import time
class SortableByCreationTime(object):
    def __init__(self):
        self._created = time.time()

    def __lt__(self, other):
        return self._created < other._created

    def __gt__(self, other):
        return self._created > other._created

In [71]:
class MyClass(MySuperclass, SortableByCreationTime):
    pass

NameError: name 'MySuperclass' is not defined

In [80]:
class Task(object):
    """A trivial task class. Task classes have a 'run': method,
    which runs the task."""
    def __call__(self, *args, **kwargs):
        return self.run(*args, **kwargs)

    def run(self, *args, **kwargs):
        raise NotImplementedError("Subclasses must implement 'run'")

    def identify(self):
        return 'I am a task.'

def task(decorated):
    """Return a class that runs the given function if its run method is called."""
    class TaskSubclass(Task):
        def run(self, *args, **kwargs):
            return decorated(*args, **kwargs)
    return TaskSubclass()

In [81]:
@task
def foo():
    return 2 + 2

In [82]:
f = foo()

In [86]:
foo.run()

4

In [85]:
foo.identify()

'I am a task.'

In [77]:
@task
def foo():
    return 2 + 2

In [78]:
foo()

<__main__.task.<locals>.TaskSubclass at 0x1e4eecd18c8>